In [1]:
import os
import dotenv

dotenv.load_dotenv()
from smolagents import CodeAgent, ToolCallingAgent, AzureOpenAIServerModel, DuckDuckGoSearchTool, tool, ActionStep

MODEL = AzureOpenAIServerModel(
    model_id=os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_VERSION")
)

MODEL_CLEVER = AzureOpenAIServerModel(
    model_id="gpt-4o-2024-11-20",
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
    api_key=os.environ.get("AZURE_OPENAI_KEY"),
    api_version=os.environ.get("AZURE_OPENAI_VERSION")
)

/private/var/user/src/edu/hf_agents/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from PIL import Image
import requests
from io import BytesIO
from IPython.display import display

image_urls = [
    "https://upload.wikimedia.org/wikipedia/commons/e/e8/The_Joker_at_Wax_Museum_Plus.jpg", # Joker image
    "https://upload.wikimedia.org/wikipedia/en/9/98/Joker_%28DC_Comics_character%29.jpg" # Joker image
]

images = []
for url in image_urls:
    response = requests.get(url)
    image = Image.open(BytesIO(response.content)).convert("RGB")
    images.append(image)
    display(image)

In [ ]:
agent = ToolCallingAgent(
    tools=[],
    model=MODEL_CLEVER,
    max_steps=20,
    verbosity_level=2
)

response = agent.run(
    """
    Describe the costume and makeup that the comic character in these photos is wearing and return the description.
    Tell me if the guest is The Joker or Wonder Woman.
    """,
    images=images
)

## Web Surfing

In [2]:
from io import BytesIO
from time import sleep

import helium
from PIL import Image
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


## Initialize the Selenium WebDriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--force-device-scale-factor=1")
chrome_options.add_argument("--window-size=1000,1350")
chrome_options.add_argument("--disable-pdf-viewer")
chrome_options.add_argument("--window-position=0,0")
driver = helium.start_chrome(headless=False, options=chrome_options)


@tool
def search_item_ctrl_f(text: str, nth_result: int = 1) -> str:
    """
    Searches for text on the current page via Ctrl + F and jumps to the nth occurrence.
    Args:
        text: The text to search for
        nth_result: Which occurrence to jump to (default: 1)
    """
    elements = driver.find_elements(By.XPATH, f"//*[contains(text(), '{text}')]")
    if nth_result > len(elements):
        raise Exception(f"Match n°{nth_result} not found (only {len(elements)} matches found)")
    result = f"Found {len(elements)} matches for '{text}'."
    elem = elements[nth_result - 1]
    driver.execute_script("arguments[0].scrollIntoView(true);", elem)
    result += f"Focused on element {nth_result} of {len(elements)}"
    return result


@tool
def go_back() -> None:
    """Goes back to previous page."""
    driver.back()


@tool
def close_popups() -> str:
    """
    Closes any visible modal or pop-up on the page. Use this to dismiss pop-up windows! This does not work on cookie consent banners.
    """
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()


def save_screenshot(step_log: ActionStep, agent: CodeAgent) -> None:
    sleep(1.0)  # Let JavaScript animations happen before taking the screenshot
    driver = helium.get_driver()
    current_step = step_log.step_number
    if driver is not None:
        for step_logs in agent.logs:  # Remove previous screenshots from logs for lean processing
            if isinstance(step_log, ActionStep) and step_log.step_number <= current_step - 2:
                step_logs.observations_images = None
        png_bytes = driver.get_screenshot_as_png()
        image = Image.open(BytesIO(png_bytes))
        print(f"Captured a browser screenshot: {image.size} pixels")
        step_log.observations_images = [image.copy()]  # Create a copy to ensure it persists, important!

    # Update observations with current URL
    url_info = f"Current url: {driver.current_url}"
    step_log.observations = url_info if step_logs.observations is None else step_log.observations + "\n" + url_info
    return

In [4]:
prompt= """
I am Alfred, the butler of Wayne Manor, responsible for verifying the identity of guests at party. A superhero has arrived at the entrance claiming to be Wonder Woman, but I need to confirm if she is who she says she is.
Please search for images of Wonder Woman and generate a detailed visual description based on those images. Additionally, navigate to Wikipedia to gather key details about her appearance. With this information, I can determine whether to grant her access to the event.
"""

helium_instructions = """
Use your web_search tool when you want to get Google search results.
Then you can use helium to access websites. Don't use helium for Google search, only for navigating websites!
Don't bother about the helium driver, it's already managed.
We've already ran "from helium import *"
Then you can go to pages!
Code:
```py
go_to('github.com/trending')
```<end_code>
You can directly click clickable elements by inputting the text that appears on them.
Code:
```py
click("Top products")
```<end_code>
If it's a link:
Code:
```py
click(Link("Top products"))
```<end_code>
If you try to interact with an element and it's not found, you'll get a LookupError.
In general stop your action after each button click to see what happens on your screenshot.
Never try to login in a page.
To scroll up or down, use scroll_down or scroll_up with as an argument the number of pixels to scroll from.
Code:
```py
scroll_down(num_pixels=1200) # This will scroll one viewport down
```<end_code>
When you have pop-ups with a cross icon to close, don't try to click the close icon by finding its element or targeting an 'X' element (this most often fails).
Just use your built-in tool `close_popups` to close them:
Code:
```py
close_popups()
```<end_code>
You can use .exists() to check for the existence of an element. For example:
Code:
```py
if Text('Accept cookies?').exists():
    click('I accept')
```<end_code>
Proceed in several steps rather than trying to solve the task in one shot.
And at the end, only when you have your answer, return your final answer.
Code:
```py
final_answer("YOUR_ANSWER_HERE")
```<end_code>
If pages seem stuck on loading, you might have to wait, for instance `import time` and run `time.sleep(5.0)`. But don't overuse this!
To list elements on page, DO NOT try code-based element searches like 'contributors = find_all(S("ol > li"))': just look at the latest screenshot you have and read it visually, or use your tool search_item_ctrl_f.
Of course, you can act on buttons like a user would do when navigating.
After each code blob you write, you will be automatically provided with an updated screenshot of the browser and the current browser url.
But beware that the screenshot will only be taken at the end of the whole action, it won't see intermediate states.
Don't kill the browser.
When you have modals or cookie banners on screen, you should get rid of them before you can click anything else.
"""

agent = CodeAgent(
    tools=[DuckDuckGoSearchTool(), go_back, close_popups, search_item_ctrl_f],
    model=MODEL_CLEVER,
    additional_authorized_imports=["helium"],
    step_callbacks=[save_screenshot],
    max_steps=20,
    verbosity_level=2,
)

# Run the agent with the provided prompt
agent.python_executor("from helium import *", agent.state)
agent.run(prompt + helium_instructions)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I am Alfred, the butler of Wayne Manor, responsible for verifying the identity of guests at party. A superhero  │
│ has arrived at the entrance claiming to be Wonder Woman, but I need to confirm if she is who she says she is.   │
│ Please search for images of Wonder Woman and generate a detailed visual description based on those images.      │
│ Additionally, navigate to Wikipedia to gather key details about her appearance. With this information, I can    │
│ determine whether to grant her access to the event.                                                             │
│                                                                                                                 │
│ Use your web_search tool when you want to get Google search results.                                            │
│ Then you can use helium to access websites. Don't use helium for Google search, only for navigating websites!   │
│ Don't bother about the helium driver, it's already managed.                                                     │
│ We've already ran "from helium import *"                                                                        │
│ Then you can go to pages!                                                                                       │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ go_to('github.com/trending')                                                                                    │
│ ```<end_code>                                                                                                   │
│ You can directly click clickable elements by inputting the text that appears on them.                           │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ click("Top products")                                                                                           │
│ ```<end_code>                                                                                                   │
│ If it's a link:                                                                                                 │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ click(Link("Top products"))                                                                                     │
│ ```<end_code>                                                                                                   │
│ If you try to interact with an element and it's not found, you'll get a LookupError.                            │
│ In general stop your action after each button click to see what happens on your screenshot.                     │
│ Never try to login in a page.                                                                                   │
│ To scroll up or down, use scroll_down or scroll_up with as an argument the number of pixels to scroll from.     │
│ Code:                                                                                                           │
│ ```py                                                                                                           │
│ scroll_down(num_pixels=1200) # This will scroll one viewport down                                               │
│ ```<end_code>                                         

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output message of the LLM: ────────────────────────────────────────────────────────────────────────────────────────
Understood. I will begin by searching for images of Wonder Woman through a web search to analyze her appearance    
visually. Then, I will navigate to Wikipedia to gather key details about her physical appearance and               
characteristics. Let's proceed step by step.                                                                       
                                                                                                                   
### Step 1: Search for Images of Wonder Woman                                                                      
Thought: I will use the `web_search` tool to find images of Wonder Woman to observe her visual appearance.         
                                                                                                                   
Code:                                                                                                              
```py                                                                                                              
image_search_results = web_search(query="Wonder Woman images")                                                     
print(image_search_results)                                                                                        
```                                                                                                                

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  image_search_results = web_search(query="Wonder Woman images")                                                   
  print(image_search_results)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Awesome Wonder Woman 4k Wallpapers - WallpaperAccess](https://wallpaperaccess.com/wonder-woman-4k)
A collection of the top 52 Wonder Woman 4k wallpapers and backgrounds available for download for free. We hope you 
enjoy our growing collection of HD images to use as a background or home screen for your smartphone or computer.

[Wonder Woman Wallpapers - Top Free Wonder Woman Backgrounds ...](https://wallpaperaccess.com/wonder-girl)
A collection of the top 78 Wonder Woman wallpapers and backgrounds available for download for free. We hope you 
enjoy our growing collection of HD images to use as a background or home screen for your smartphone or computer. 
Please contact us if you want to publish a Wonder Woman wallpaper on our site.

[7,088 Wonder Woman Images Stock Photos & High-Res Pictures - Getty 
Images](https://www.gettyimages.com/photos/wonder-woman-images)
usa wonder woman comic book cover postage stamp - wonder woman images stock pictures, royalty-free photos & images.
USA Wonder Woman comic book cover postage stamp. Fan cosplays as Wonder Woman from the DC universe during 2017 New 
York Comic Con - Day 2 on October 6, 2017 in New York City.

[Wonder Woman 4K Wallpapers - Wallpaper Cave](https://wallpapercave.com/wonder-woman-4k-wallpapers)
Tons of awesome Wonder Woman 4K wallpapers to download for free. You can also upload and share your favorite Wonder
Woman 4K wallpapers. HD wallpapers and background images

[100,000+ Free Wonder Woman & Woman Images - Pixabay](https://pixabay.com/images/search/wonder+woman/)
183,947 Free images of Wonder Woman. Wonder woman images for free download. Browse or use the filters to find your 
next picture for your project.

[[1100+] Wonder Woman Wallpapers - Alpha Coders](https://alphacoders.com/wonder-woman-wallpapers)
Wonder Woman Wallpapers Favorite Infinite Pages Best More New. Rating. Views. All Resolutions At least Exactly. All
Resolutions 2560x1440 3840x2160 5120x2880 7680x4320. Custom: X Submit [1100+] Experience the power and beauty of 
Wonder Woman in stunning HD with our vast collection of desktop wallpapers that will transport you into the heart 
of ...

[[200+] Wonder Woman Wallpapers | Wallpapers.com](https://wallpapers.com/wonder-woman)
Download Wonder Woman Wallpapers Get Free Wonder Woman Wallpapers in sizes up to 8K 100% Free Download & 
Personalise for all Devices. ... Show off your love for this iconic superhero with unique and vibrant images 
perfect for your mobile or computer screen. Wonder Woman Wallpapers Wallpapers; Wallpaper Folders;

[Wonder Woman screenshots, images and pictures - Comic 
Vine](https://comicvine.gamespot.com/wonder-woman/4005-2048/images/)
Browse 4622 screenshots, images and pictures of Wonder Woman, the Amazon princess and Justice League member. See 
her origin, powers, and appearances in comics and other media.

[Wonder Woman Photos, Download The BEST Free Wonder Woman Stock Photos 
...](https://www.pexels.com/search/wonder+woman/)
Download and use 700,000+ Wonder Woman stock photos for free. Thousands of new images every day Completely Free to 
Use High-quality videos and images from Pexels. Photos. Explore. License. Upload Join. Free Wonder Woman Photos. 
Photos 744.3K Videos 139.8K Users 2K. Filters. Popular.

[Category:Wonder Woman Gallery/Images - DC 
Database](https://dc.fandom.com/wiki/Category:Wonder_Woman_Gallery/Images)
This is our collection of Wonder Woman Gallery images. If you find an image in the database that is not shown here,
please edit that image adding "Wonder Woman Gallery" to the Issue field. Trending pages. Batman (Bruce Wayne) 
Superman (Clark Kent) Justice League (Prime Earth) Lantern Corps ...

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 9 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

The 'logs' attribute is deprecated and will soon be removed. Please use 'self.memory.steps' instead.


Captured a browser screenshot: (1000, 805) pixels


[Step 8: Duration 0.00 seconds| Input tokens: 59,093 | Output tokens: 758]

TypeError: 'NoneType' object is not subscriptable